# Create a JSON/list of dicts setup of foods and their metadata

For example:

```
{
    "food_items": [
        {
            "nutrify_name": "apple",
            "other_names": ["malus domestica"],
            "food_id": "F00001",
            "various_spellings": ["appel", "aplpe", "aple"],
            "display_name": "Apple"
        },
        {
            "nutrify_name": "banana",
            "other_names": ["musa sapientum"],
            "food_id": "F00002",
            "various_spellings": ["bannana", "banan"],
            "display_name": "Banana"
        }
    ]
}
```

Doing this will mean that the foods can be updated and accessed via various names but their display name will still show.

For example, "iceberg_lettuce" will still be recognized via "lettuce_iceberg" and will display as "Iceberg Lettuce".

In [1]:
import openai
# openai.__version__

In [2]:
# Append the upper level directory to sys
import sys
sys.path.append("..")

import pandas as pd
import numpy as np

from pathlib import Path

import torch

from tqdm.auto import tqdm

from utils.misc import sort_dict_by_values

# Get original labels to label new data from
# Get config
from configs.default_config import config

args = config

# Connect to GCP
from utils.gcp_utils import set_gcp_credentials, test_gcp_connection
set_gcp_credentials(path_to_key="../utils/google-storage-key.json")
test_gcp_connection()

import wandb

# Initialize a new run
from utils.wandb_utils import wandb_load_artifact, wandb_download_and_load_labels

notes = f"merge vegfru dataset with Nutrify"

run = wandb.init(project=args.wandb_project, 
                 job_type=args.wandb_job_type,
                 tags=['auto_label_new_images'],
                 notes=notes)

annotations, class_names, class_dict, reverse_class_dict, labels_path = wandb_download_and_load_labels(wandb_run=run,
wandb_labels_artifact_name=args.wandb_labels_artifact)

len(annotations.label.unique())

[INFO] GCP credentials set!
[INFO] GCP connection successful! Access to GCP for saving/loading data and models available.


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: mrdbourke. Use `wandb login --relogin` to force relogin


[INFO] Labels directory: ./artifacts/food_vision_labels:v82
[INFO] Labels path: artifacts/food_vision_labels:v82/annotations.csv
[INFO] Working with: 361 classes


361

In [3]:
class_dict

{0: 'achacha',
 1: 'almond_butter',
 2: 'almonds',
 3: 'apple_custard',
 4: 'apple_green',
 5: 'apple_red',
 6: 'apricot',
 7: 'artichoke',
 8: 'asparagus',
 9: 'avocado',
 10: 'bacon',
 11: 'bacon_and_egg_burger',
 12: 'bagel',
 13: 'baklava',
 14: 'banana',
 15: 'banana_bread',
 16: 'barbecue_sauce',
 17: 'basil',
 18: 'bay_leaves',
 19: 'bean_snake',
 20: 'bean_sprouts',
 21: 'beans',
 22: 'beef_corned',
 23: 'beef_curry',
 24: 'beef_diced',
 25: 'beef_kebab',
 26: 'beef_mince',
 27: 'beef_patty',
 28: 'beef_roast',
 29: 'beef_stir_fry',
 30: 'beer',
 31: 'beetroot',
 32: 'biltong',
 33: 'biscuits',
 34: 'black_pepper',
 35: 'blackberries',
 36: 'blueberries',
 37: 'bok_choy',
 38: 'brazil_nuts',
 39: 'bread',
 40: 'bread_naan',
 41: 'breadfruit',
 42: 'broccoli',
 43: 'broccolini',
 44: 'brownie',
 45: 'brussel_sprouts',
 46: 'burrito',
 47: 'butter',
 48: 'cabbage_green',
 49: 'cabbage_red',
 50: 'cabbage_savoy',
 51: 'calamari',
 52: 'candy',
 53: 'cape_gooseberries',
 54: 'capsi

In [4]:
# Import food_class_dicts.json
import json

# Copy the food_class_metadata_dicts.json from Google Storage (TODO: could add this to WANDB artifacts later on)
!gsutil cp gs://food_vision_bucket_with_object_versioning/nutrition_information/nutrify_foodvision_items_nutrients.json ./nutrify_foodvision_items_nutrients.json
!gsutil cp gs://food_vision_bucket_with_object_versioning/nutrition_information/food_class_metadata_dicts.json ./food_class_metadata_dicts.json

# Load the food_class_metadata_dicts.json
food_class_dicts_path = "food_class_metadata_dicts.json"
food_class_dicts = json.load(open(food_class_dicts_path, "r"))
food_class_dicts[:3]



Updates are available for some Google Cloud CLI components.  To install them,
please run:
  $ gcloud components update

Copying gs://food_vision_bucket_with_object_versioning/nutrition_information/nutrify_foodvision_items_nutrients.json...
- [1 files][407.7 KiB/407.7 KiB]                                                
Operation completed over 1 objects/407.7 KiB.                                    
Copying gs://food_vision_bucket_with_object_versioning/nutrition_information/food_class_metadata_dicts.json...
- [1 files][  1.2 MiB/  1.2 MiB]                                                
Operation completed over 1 objects/1.2 MiB.                                      


[{'nutrify_name': 'almond_butter',
  'id': 0,
  'other_names': ['butter_almond', 'almond butter', 'Almond Butter'],
  'display_name': 'Almond Butter',
  'nutrition_information': {'fdcId': 2262074,
   'description': 'Almond butter, creamy',
   'default_size': 100,
   'unit': 'g',
   'total_energy_calories': 645,
   'micronutrients': {'vitamins': [{'name': 'Thiamin',
      'amount': 0.095,
      'unit': 'mg'},
     {'name': 'Niacin', 'amount': 3.91, 'unit': 'mg'},
     {'name': 'Vitamin B-6', 'amount': 0.084, 'unit': 'mg'},
     {'name': 'Biotin', 'amount': 57.3, 'unit': 'µg'},
     {'name': 'Folate, total', 'amount': 42.5, 'unit': 'µg'},
     {'name': 'Vitamin E (alpha-tocopherol)', 'amount': 16.7, 'unit': 'mg'},
     {'name': 'Vitamin K (phylloquinone)', 'amount': 0, 'unit': 'µg'}],
    'minerals': [{'name': 'Calcium, Ca', 'amount': 264, 'unit': 'mg'},
     {'name': 'Iron, Fe', 'amount': 4.11, 'unit': 'mg'},
     {'name': 'Magnesium, Mg', 'amount': 268, 'unit': 'mg'},
     {'name': 'Ph

In [5]:
import openai

openai.api_key_path = "../utils/openai_api_key.txt"

def chat_complete(prompt: str, **openai_kwargs) -> str:
    """Generate completion from OpenAI ChatGPT API"""
    default_kwargs = {"model": "gpt-3.5-turbo", "max_tokens": 256, "temperature": 0.2, "top_p": 1}
    openai_kwargs = {**default_kwargs, **openai_kwargs}
    response = openai.ChatCompletion.create(
        **openai_kwargs,
        messages=[
                {"role": "system", "content": "You are a helpful assistant who is very good at designing apps and making them look good."},
                {"role": "user", "content": prompt},
            ]
        )
    return response

base_prompt_for_display_name_creation = """You are an app designer trying to make your app as appealing as possible to people who use it.\n
You've been tasked with making food names that appear in a database as visually appealing as possible.\n
Given the target text '{target_text}' you want to turn it into the most grammatically correct way to display it as a title on an app screen.\n
The name should reflect how most people would refer to the food.\n\nYou should correct the order if necessary.\n
For example: \n- \"apple_green\" would appear as \"Green Apple\"\n- \"potato_roast\" would appear as \"Roast Potatoes\"\n- \"tomato_cherry\" would appear as \"Cherry Tomatoes\"\n- \"lettuce_cos\" would appear as \"Cos Lettuce\"\n
No extra words should be added to the target text.\n
For example: \n- \"mint\" would become \"Mint\" (no extra words)\n- \"cookies\" would become \"Cookies\"\n- \"cheese\" would become \"Cheese\"\n- \"beans\" would become \"Beans\"\n
Please do not provide any reasoning at all, just the corrected text.\n
Answer (grammatically correct way of showing the target text '{target_text}'):
"""

def generate_display_name(target_text: str) -> str:
    target_text_prompt = base_prompt_for_display_name_creation.format(target_text=target_text)
    answer = chat_complete(target_text_prompt)
    answer_formatted = str(answer.choices[0].message.content).strip().replace('"', "").replace("'", "")
    print(f"Display name generated: {target_text} -> {answer_formatted}")
    return answer_formatted

target_text = "apple_green"
generate_display_name(target_text)

Display name generated: apple_green -> Green Apple


'Green Apple'

In [6]:
existing_food_class_dicts = [food_class_dict["nutrify_name"] for food_class_dict in food_class_dicts] 

# TODO
# remove the food_class_metadata_dicts if they're "nutrify_name" not in class_names

new_display_names_generated = []
for class_name in class_names:
    if class_name in existing_food_class_dicts:
        print(f"Metadata dict for class name '{class_name}' exists")
        target_food_dict = [food_class_dict for food_class_dict in food_class_dicts if food_class_dict["nutrify_name"] == class_name][0]

        try:
            target_food_display_name = target_food_dict["display_name"]
        except KeyError:
            target_food_display_name = None

        if target_food_display_name == None:
            print(f"Display name for class name '{class_name}' is None, generating...")
            # TODO: turn this into a function
            display_name = generate_display_name(class_name)
            target_food_dict["display_name"] = display_name 
            new_display_names_generated.append(target_food_dict)
        else:
            print(f"Display name for class name '{class_name}' is '{target_food_display_name}'")
            
    else:
        print(f"Creating metadata dict for class name '{class_name}'")
        food_dict = {}
        food_dict["nutrify_name"] = class_name
        # TODO: change this to some kind of UUID?
        food_dict["id"] = class_names.index(class_name)

        # Reverse the order of the words in the class name
        # e.g. "apple_pie" -> "pie_apple"
        other_names = []
        other_names.append(class_name) # original name
        other_names.append("_".join(class_name.split("_")[::-1])) # reversed name
        other_names.append(class_name.replace("_", " ")) # space separated
        other_names.append(class_name.replace("_", " ").title()) # space separated, title case

        # TODO: generate possible other names for different foods (e.g. Eggplant -> Aubergine, Pain au Chocolate -> Chocolate Croissant)
        food_dict["other_names"] = other_names
        
        # TODO: Generate a display name if the display name doesn't exist
        food_dict["display_name"] = generate_display_name(class_name)
        food_class_dicts.append(food_dict)
        new_display_names_generated.append(food_dict) # Check for new display names generated

food_class_dicts

Metadata dict for class name 'achacha' exists
Display name for class name 'achacha' is 'Achacha'
Metadata dict for class name 'almond_butter' exists
Display name for class name 'almond_butter' is 'Almond Butter'
Metadata dict for class name 'almonds' exists
Display name for class name 'almonds' is 'Almonds'
Metadata dict for class name 'apple_custard' exists
Display name for class name 'apple_custard' is 'Custard Apple'
Metadata dict for class name 'apple_green' exists
Display name for class name 'apple_green' is 'Green Apple'
Metadata dict for class name 'apple_red' exists
Display name for class name 'apple_red' is 'Red Apple'
Metadata dict for class name 'apricot' exists
Display name for class name 'apricot' is 'Apricots'
Metadata dict for class name 'artichoke' exists
Display name for class name 'artichoke' is 'Artichoke'
Metadata dict for class name 'asparagus' exists
Display name for class name 'asparagus' is 'Asparagus'
Metadata dict for class name 'avocado' exists
Display name f

[{'nutrify_name': 'almond_butter',
  'id': 0,
  'other_names': ['butter_almond', 'almond butter', 'Almond Butter'],
  'display_name': 'Almond Butter',
  'nutrition_information': {'fdcId': 2262074,
   'description': 'Almond butter, creamy',
   'default_size': 100,
   'unit': 'g',
   'total_energy_calories': 645,
   'micronutrients': {'vitamins': [{'name': 'Thiamin',
      'amount': 0.095,
      'unit': 'mg'},
     {'name': 'Niacin', 'amount': 3.91, 'unit': 'mg'},
     {'name': 'Vitamin B-6', 'amount': 0.084, 'unit': 'mg'},
     {'name': 'Biotin', 'amount': 57.3, 'unit': 'µg'},
     {'name': 'Folate, total', 'amount': 42.5, 'unit': 'µg'},
     {'name': 'Vitamin E (alpha-tocopherol)', 'amount': 16.7, 'unit': 'mg'},
     {'name': 'Vitamin K (phylloquinone)', 'amount': 0, 'unit': 'µg'}],
    'minerals': [{'name': 'Calcium, Ca', 'amount': 264, 'unit': 'mg'},
     {'name': 'Iron, Fe', 'amount': 4.11, 'unit': 'mg'},
     {'name': 'Magnesium, Mg', 'amount': 268, 'unit': 'mg'},
     {'name': 'Ph

In [7]:
# Remove the food_class_metadata_dicts if they're "nutrify_name" not in class_names
print(f"Length of food_class_dicts before removal: {len(food_class_dicts)}")
food_class_dicts = [food_class_dict for food_class_dict in food_class_dicts if food_class_dict["nutrify_name"] in class_names]
print(f"Length of food_class_dicts after removal: {len(food_class_dicts)}")

Length of food_class_dicts before removal: 361
Length of food_class_dicts after removal: 361


In [8]:
# Get the filesize of the food_class_metadata_dicts.json file
import os
os.path.getsize(food_class_dicts_path)

1202864

In [9]:
len(new_display_names_generated), new_display_names_generated

(0, [])

In [10]:
# Save the food class dicts to a JSON file
with open("food_class_metadata_dicts.json", "w") as f:
    json.dump(food_class_dicts, f, indent=4)

In [11]:
!gsutil cp gs://food_vision_bucket_with_object_versioning/nutrition_information/nutrify_foodvision_items_nutrients.json ./nutrify_foodvision_items_nutrients.json

Copying gs://food_vision_bucket_with_object_versioning/nutrition_information/nutrify_foodvision_items_nutrients.json...
- [1 files][407.7 KiB/407.7 KiB]                                                
Operation completed over 1 objects/407.7 KiB.                                    


In [12]:
# Load the nutrify_foodvision_items_nutrients.json file
import json

# nutrify_foodvision_items_nutrients = json.load("nutrify_foodvision_items_nutrients.json")

with open("nutrify_foodvision_items_nutrients.json", "r") as f:
    nutrify_foodvision_items_nutrients = json.load(f)

# Load back in the food_class_metadata_dicts.json file
with open("food_class_metadata_dicts.json", "r") as f:
    food_class_dicts = json.load(f)

In [13]:
# Get the map from nutrify_name to FDC ID
import gspread
import pandas as pd

# Open the sheet with Google Sheets
gc = gspread.oauth()

# See the link here: https://docs.google.com/spreadsheets/d/1fgNtqfuaBSbPBPQfyzYAmRb1UUHdsRphoAqXHfP-dD0/edit?usp=sharing 
sheet = gc.open("nutrify-name-to-fdc-id")
worksheet = sheet.sheet1
rows = worksheet.get_all_records()

nutrify_to_fdcid = pd.DataFrame(rows)

# Replace all blank rows in fdc_id with NaN
nutrify_to_fdcid['fdc_id'] = nutrify_to_fdcid['fdc_id'].replace('', None)
nutrify_to_fdcid.head()

,number,class_name,fdc_id
0,0,achacha,None
1,1,almond_butter,2262074
2,2,almonds,2346393
3,3,apple_custard,168175
4,4,apple_green,1750342


In [14]:
# Get the class_name to fdc_id as a dict
class_name_to_fdc_id = dict(zip(nutrify_to_fdcid["class_name"], nutrify_to_fdcid["fdc_id"]))
class_name_to_fdc_id

{'achacha': None,
 'almond_butter': 2262074,
 'almonds': 2346393,
 'apple_custard': 168175,
 'apple_green': 1750342,
 'apple_red': 1750339,
 'apricot': 171697,
 'artichoke': 169205,
 'asparagus': 168389,
 'avocado': 171705,
 'bacon': 749420,
 'bacon_and_egg_burger': None,
 'bagel': 2343212,
 'baklava': 2343527,
 'banana': 1105314,
 'banana_bread': None,
 'barbecue_sauce': 174523,
 'basil': 172232,
 'bay_leaves': 170917,
 'bean_sprouts': 2345288,
 'beans': 747444,
 'beef_curry': 2341807,
 'beef_diced': None,
 'beef_kebab': None,
 'beef_mince': 2341259,
 'beef_patty': 2341260,
 'beef_roast': 2341252,
 'beef_stir_fry': 2342187,
 'beer': 2346192,
 'beetroot': 2345290,
 'biltong': None,
 'black_pepper': None,
 'blackberries': 2344767,
 'blueberries': 2346411,
 'bok_choy': 2345294,
 'bread': 335240,
 'bread_naan': 2343090,
 'broccoli': 747447,
 'broccolini': 2345073,
 'brownie': 2343384,
 'brussel_sprouts': 2345292,
 'burrito': 2344031,
 'butter': 790508,
 'cabbage_green': 2346407,
 'cabbage

In [15]:
nutrify_foodvision_items_nutrients[0]["fdcId"]

321360

In [22]:
for item in tqdm(food_class_dicts):
    class_name = item["nutrify_name"]
    try:
        fdc_id = class_name_to_fdc_id[class_name]
        if fdc_id == None:
            item["nutrition_information"] = {}
        else:
            for foodvision_nutrition_item in nutrify_foodvision_items_nutrients:
                if foodvision_nutrition_item["fdcId"] == fdc_id:
                    item["nutrition_information"] = foodvision_nutrition_item
    except KeyError:
        fdc_id = None
        item["nutrition_information"] = {}
    # item["fdc_id"] = fdc_id

  0%|          | 0/361 [00:00<?, ?it/s]

In [23]:
len(food_class_dicts)

361

In [24]:
items_with_no_nutrition_information = []
for item in food_class_dicts:
    # print(f"on item: {item['nutrify_name']}")
    if item["nutrition_information"] == {}:
        items_with_no_nutrition_information.append(item)

In [25]:
len(items_with_no_nutrition_information)

81

In [26]:
items_with_no_nutrition_information

[{'nutrify_name': 'bacon_and_egg_burger',
  'id': 9,
  'other_names': ['burger_egg_and_bacon',
   'bacon and egg burger',
   'Bacon And Egg Burger'],
  'display_name': 'Bacon and Egg Burger',
  'nutrition_information': {}},
 {'nutrify_name': 'banana_bread',
  'id': 13,
  'other_names': ['bread_banana', 'banana bread', 'Banana Bread'],
  'display_name': 'Banana Bread',
  'nutrition_information': {}},
 {'nutrify_name': 'beef_diced',
  'id': 19,
  'other_names': ['diced_beef', 'beef diced', 'Beef Diced'],
  'display_name': 'Diced Beef',
  'nutrition_information': {}},
 {'nutrify_name': 'beef_kebab',
  'id': 20,
  'other_names': ['kebab_beef', 'beef kebab', 'Beef Kebab'],
  'display_name': 'Beef Kebab',
  'nutrition_information': {}},
 {'nutrify_name': 'biltong',
  'id': 27,
  'other_names': ['biltong', 'biltong', 'Biltong'],
  'display_name': 'Biltong',
  'nutrition_information': {}},
 {'nutrify_name': 'chimichurri',
  'id': 59,
  'other_names': ['chimichurri', 'chimichurri', 'Chimichurri

In [27]:
# Save food_class_dicts to a JSON file
with open("food_class_metadata_dicts.json", "w", encoding="utf-8") as f:
    json.dump(food_class_dicts, f, indent=4, ensure_ascii=False)

# Upload the food_class_metadata_dicts.json file to Google Cloud Storage
!gsutil cp food_class_metadata_dicts.json gs://food_vision_bucket_with_object_versioning/nutrition_information/food_class_metadata_dicts.json

Copying file://food_class_metadata_dicts.json [Content-Type=application/json]...
\ [1 files][  1.2 MiB/  1.2 MiB]                                                
Operation completed over 1 objects/1.2 MiB.                                      


In [ ]:
# Download the nutrition information JSON and interleave it with food class metadata
# Download from Google Storage
# Go through each food class dict and add the nutrition information to it (based on the matching nutrify name)

In [5]:
import openai

openai.api_key_path = "../utils/openai_api_key.txt"

In [6]:
prompt = "With the text {} please generate the most grammatically correct way to display it as a title on an app screen.\n\nFor example, the text \"apple_green\" would appear as \"Green Apple\".\n\nThe name should reflect how most people would refer to the food.\n\nFor example: \n- \"potato_roast\" would appear as \"Roast Potatoes\".\n- \"tomato_cherry\" would appear as \"Cherry Tomato\".\n\nAnd it should correct the order if necessary.\n\nFor example, \"lettuce_cos\" would be \"Cos Lettuce\".\n\nPlease do not add any more words to what is existing, only rearrange the existing text.\n\nFor example, \"mint\" would become \"Mint\" (no extra words).\n\nAnswer:"

prompt_formatted = prompt.format("apple_green")

prompt_formatted

'With the text apple_green please generate the most grammatically correct way to display it as a title on an app screen.\n\nFor example, the text "apple_green" would appear as "Green Apple".\n\nThe name should reflect how most people would refer to the food.\n\nFor example: \n- "potato_roast" would appear as "Roast Potatoes".\n- "tomato_cherry" would appear as "Cherry Tomato".\n\nAnd it should correct the order if necessary.\n\nFor example, "lettuce_cos" would be "Cos Lettuce".\n\nPlease do not add any more words to what is existing, only rearrange the existing text.\n\nFor example, "mint" would become "Mint" (no extra words).\n\nAnswer:'

In [7]:
response = openai.Completion.create(
  model="text-davinci-003",
  prompt=prompt_formatted,
  temperature=0,
  max_tokens=256,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)

response.choices[0].text.strip()

'"Apple Green"'

In [25]:
# Note: you need to be using OpenAI Python v0.27.0 for the code below to work
import openai

chat_message = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "system", "content": "You are a helpful assistant who is very good at designing apps and making them look good."},
        {"role": "user", "content": prompt_formatted},
    ],
  temperature=0,
  max_tokens=256,
)

chat_message

<OpenAIObject chat.completion id=chatcmpl-6rGAWEvkeoJ3mgtul8YvGt43wHH1l at 0x7fec119eb4a0> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "Green Apple",
        "role": "assistant"
      }
    }
  ],
  "created": 1678151556,
  "id": "chatcmpl-6rGAWEvkeoJ3mgtul8YvGt43wHH1l",
  "model": "gpt-3.5-turbo-0301",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 4,
    "prompt_tokens": 181,
    "total_tokens": 185
  }
}

In [27]:
chat_message.choices[0].message.content

'Green Apple'

In [37]:
# from tqdm.auto import tqdm
# for food_class_dict in tqdm(food_class_dicts):
#     prompt_formatted = prompt.format(food_class_dict["nutrify_name"])
#     response = openai.Completion.create(
#       model="text-davinci-003",
#       prompt=prompt_formatted,
#       temperature=0,
#       max_tokens=256,
#       top_p=1,
#       frequency_penalty=0,
#       presence_penalty=0
#     )
#     food_class_dict["display_name"] = response.choices[0].text.strip()

  0%|          | 0/259 [00:00<?, ?it/s]

In [13]:
display_name_to_food_class_dict = {}
for food_class_dict in food_class_dicts:
    food_class_dict["display_name"] = food_class_dict["display_name"].replace('"', "")
    display_name_to_food_class_dict[food_class_dict["nutrify_name"]] = food_class_dict["display_name"]
display_name_to_food_class_dict

{'almond_butter': 'Almond Butter',
 'almonds': 'Almonds',
 'apple_green': 'Apple Green',
 'apple_red': 'Apple Red',
 'apricot': 'Apricot',
 'artichoke': 'Artichoke',
 'asparagus': 'Asparagus',
 'avocado': 'Avocado',
 'bacon': 'Bacon',
 'bacon_and_egg_burger': 'Bacon And Egg Burger',
 'bagel': 'Bagel',
 'baklava': 'Baklava',
 'banana': 'Banana',
 'banana_bread': 'Banana Bread',
 'barbecue_sauce': 'Barbecue Sauce',
 'basil': 'Basil',
 'bean_sprouts': 'Bean Sprouts',
 'beans': 'Beans',
 'beef_curry': 'Beef Curry',
 'beef_diced': 'Beef Diced',
 'beef_kebab': 'Beef Kebab',
 'beef_mince': 'Beef Mince',
 'beef_patty': 'Beef Patty',
 'beef_roast': 'Beef Roast',
 'beef_stir_fry': 'Beef Stir Fry',
 'beer': 'Beer',
 'beetroot': 'Beetroot',
 'biltong': 'Biltong',
 'blackberries': 'Blackberries',
 'blueberries': 'Blueberries',
 'bok_choy': 'Bok Choy',
 'bread': 'Bread',
 'bread_naan': 'Bread Naan',
 'broccoli': 'Broccoli',
 'broccolini': 'Broccolini',
 'brownie': 'Brownie',
 'brussel_sprouts': 'Bru

In [ ]:
# To change
# beans -> beans
# cabbage_green -> Green Cabbage
# cabbage_red -> Red Cabbage
# cheese -> Cheese
# cookies -> Cookies
# pasta -> Pasta
# peas -> Peas
# sandwich -> Sandwich


### Try Chain-of-Thought

In [35]:
def chat_complete(prompt: str, **openai_kwargs) -> str:
    """Generate completion from OpenAI ChatGPT API"""
    default_kwargs = {"model": "gpt-3.5-turbo", "max_tokens": 256, "temperature": 0.2, "top_p": 1}
    openai_kwargs = {**default_kwargs, **openai_kwargs}
    response = openai.ChatCompletion.create(
        **openai_kwargs,
        messages=[
                {"role": "system", "content": "You are a helpful assistant who is very good at designing apps and making them look good."},
                {"role": "user", "content": prompt},
            ]
        )
    return response

def complete(prompt: str, **openai_kwargs) -> str:
    """Generate completion from OpenAI API"""
    default_kwargs = {"model": "text-davinci-003", "max_tokens": 256}
    openai_kwargs = {**default_kwargs, **openai_kwargs}
    response = openai.Completion.create(prompt=prompt, **openai_kwargs)
    return response.choices[0]["text"]

def ask_chain_of_thought(question: str) -> str:
    """Generate a chain of thought for given question."""
    cot_prompt_format = "Q:{question}\nA: Let's think step by step."
    extract_prompt_format = "{cot_prompt}{cot_completion} Therefore, the final answer (grammatically correct way of showing the target text) is:"
    cot_prompt = cot_prompt_format.format(question=question)
    cot_completion = complete(cot_prompt, temperature=0, presence_penalty=0, frequency_penalty=0)
    extract_prompt = extract_prompt_format.format(cot_prompt=cot_prompt, cot_completion=cot_completion)
    extract_completion = complete(extract_prompt, temperature=0, presence_penalty=0, frequency_penalty=0)
    return extract_completion

def ask_consensus_question(question: str, n: int = 5) -> str:
    """Generate consensus answer for given question by asking OpenAI n times."""
    answers = [ask_chain_of_thought(question) for _ in range(n)]
    print(answers)
    return max(set(answers), key=answers.count)

In [36]:
# import random
# # target_text = random.choice(food_classes)
# target_text = "potato_roast"
# print(f"Target text: {target_text}")
# question = "What is the most grammatically correct way to display the text '{target_text}' as a title on an app screen?"
# question = question.format(target_text=target_text)
# answer = ask_consensus_question(question)
# print(f"Q: {question}")
# print(f"A: {answer}")

In [39]:
base_prompt = """You are an app designer trying to make your app as appealing as possible to people who use it.\n
You've been tasked with making food names that appear in a database as visually appealing as possible.\n
Given the target text '{target_text}' you want to turn it into the most grammatically correct way to display it as a title on an app screen.\n
The name should reflect how most people would refer to the food.\n\nYou should correct the order if necessary.\n
For example: \n- \"apple_green\" would appear as \"Green Apple\"\n- \"potato_roast\" would appear as \"Roast Potatoes\"\n- \"tomato_cherry\" would appear as \"Cherry Tomatoes\"\n- \"lettuce_cos\" would appear as \"Cos Lettuce\"\n
No extra words should be added to the target text.\n
For example: \n- \"mint\" would become \"Mint\" (no extra words)\n- \"cookies\" would become \"Cookies\"\n- \"cheese\" would become \"Cheese\"\n- \"beans\" would become \"Beans\"\n
Please do not provide any reasoning at all, just the corrected text.\n
Answer (grammatically correct way of showing the target text '{target_text}'):
"""

print(base_prompt.format(target_text="beans"))

You are an app designer trying to make your app as appealing as possible to people who use it.

You've been tasked with making food names that appear in a database as visually appealing as possible.

Given the target text 'beans' you want to turn it into the most grammatically correct way to display it as a title on an app screen.

The name should reflect how most people would refer to the food.

You should correct the order if necessary.

For example: 
- "apple_green" would appear as "Green Apple"
- "potato_roast" would appear as "Roast Potatoes"
- "tomato_cherry" would appear as "Cherry Tomatoes"
- "lettuce_cos" would appear as "Cos Lettuce"

No extra words should be added to the target text.

For example: 
- "mint" would become "Mint" (no extra words)
- "cookies" would become "Cookies"
- "cheese" would become "Cheese"
- "beans" would become "Beans"

Please do not provide any reasoning at all, just the corrected text.

Answer (grammatically correct way of showing the target text 'bea

In [41]:
for food_class_dict in tqdm(food_class_dicts):
    target_text = food_class_dict["nutrify_name"]
    target_text_prompt = base_prompt.format(target_text=target_text)
    answer = chat_complete(target_text_prompt)
    # print(answer)
    answer_formatted = str(answer.choices[0].message.content).strip().replace('"', "").replace("'", "")
    print(f"Target text -> answer: {target_text} -> {answer_formatted}")
    food_class_dict["display_name"] = answer_formatted

  0%|          | 0/252 [00:00<?, ?it/s]

Target text -> answer: almond_butter -> Almond Butter
Target text -> answer: almonds -> Almonds
Target text -> answer: apple_green -> Green Apple
Target text -> answer: apple_red -> Red Apple
Target text -> answer: apricot -> Apricots
Target text -> answer: artichoke -> Artichoke
Target text -> answer: asparagus -> Asparagus
Target text -> answer: avocado -> Avocado
Target text -> answer: bacon -> Bacon
Target text -> answer: bacon_and_egg_burger -> Bacon and Egg Burger
Target text -> answer: bagel -> Bagel
Target text -> answer: baklava -> Baklava
Target text -> answer: banana -> Banana
Target text -> answer: banana_bread -> Banana Bread
Target text -> answer: barbecue_sauce -> Barbecue Sauce
Target text -> answer: basil -> Basil
Target text -> answer: bean_sprouts -> Sprouted Beans
Target text -> answer: beans -> Beans
Target text -> answer: beef_curry -> Curry Beef
Target text -> answer: beef_diced -> Diced Beef
Target text -> answer: beef_kebab -> Kebab of Beef
Target text -> answe

In [42]:
display_name_to_food_class_dict = {}
for food_class_dict in food_class_dicts:
    food_class_dict["display_name"] = food_class_dict["display_name"].replace('"', "").strip()
    display_name_to_food_class_dict[food_class_dict["nutrify_name"]] = food_class_dict["display_name"]
display_name_to_food_class_dict

{'almond_butter': 'Almond Butter',
 'almonds': 'Almonds',
 'apple_green': 'Green Apple',
 'apple_red': 'Red Apple',
 'apricot': 'Apricots',
 'artichoke': 'Artichoke',
 'asparagus': 'Asparagus',
 'avocado': 'Avocado',
 'bacon': 'Bacon',
 'bacon_and_egg_burger': 'Bacon and Egg Burger',
 'bagel': 'Bagel',
 'baklava': 'Baklava',
 'banana': 'Banana',
 'banana_bread': 'Banana Bread',
 'barbecue_sauce': 'Barbecue Sauce',
 'basil': 'Basil',
 'bean_sprouts': 'Sprouted Beans',
 'beans': 'Beans',
 'beef_curry': 'Curry Beef',
 'beef_diced': 'Diced Beef',
 'beef_kebab': 'Kebab of Beef',
 'beef_mince': 'Mince Beef',
 'beef_patty': 'Patty of Beef',
 'beef_roast': 'Roast Beef',
 'beef_stir_fry': 'Stir-Fried Beef',
 'beer': 'Beer',
 'beetroot': 'Beetroot',
 'biltong': 'Biltong',
 'blackberries': 'Blackberries',
 'blueberries': 'Blueberries',
 'bok_choy': 'Bok Choy',
 'bread': 'Bread',
 'bread_naan': 'Naan Bread',
 'broccoli': 'Broccoli',
 'broccolini': 'Broccolini',
 'brownie': 'Brownies',
 'brussel_sp

In [44]:
# # Export the dictionary to a JSON file
# import json
# with open('display_name_to_food_class_dict.json', 'w') as fp:
#     json.dump(display_name_to_food_class_dict, fp)

In [57]:
# Open the JSON file and load the dictionary
import json
with open('display_name_to_food_class_dict.json', 'r') as fp:
    display_name_to_food_class_dict = json.load(fp)

In [58]:
# Go through the food_class_dict and update the display_name
for food_class_dict in food_class_dicts:
    food_class_dict["display_name"] = display_name_to_food_class_dict[food_class_dict["nutrify_name"]]

food_class_dicts

[{'nutrify_name': 'almond_butter',
  'id': 0,
  'other_names': ['butter_almond', 'almond butter', 'Almond Butter'],
  'display_name': 'Almond Butter'},
 {'nutrify_name': 'almonds',
  'id': 1,
  'other_names': ['almonds', 'almonds', 'Almonds'],
  'display_name': 'Almonds'},
 {'nutrify_name': 'apple_green',
  'id': 2,
  'other_names': ['green_apple', 'apple green', 'Apple Green'],
  'display_name': 'Green Apple'},
 {'nutrify_name': 'apple_red',
  'id': 3,
  'other_names': ['red_apple', 'apple red', 'Apple Red'],
  'display_name': 'Red Apple'},
 {'nutrify_name': 'apricot',
  'id': 4,
  'other_names': ['apricot', 'apricot', 'Apricot'],
  'display_name': 'Apricots'},
 {'nutrify_name': 'artichoke',
  'id': 5,
  'other_names': ['artichoke', 'artichoke', 'Artichoke'],
  'display_name': 'Artichoke'},
 {'nutrify_name': 'asparagus',
  'id': 6,
  'other_names': ['asparagus', 'asparagus', 'Asparagus'],
  'display_name': 'Asparagus'},
 {'nutrify_name': 'avocado',
  'id': 7,
  'other_names': ['avocad

In [59]:
# Save food_class_dicts to a JSON file
import json
with open('food_class_dicts.json', 'w') as fp:
    json.dump(food_class_dicts, fp)

In [60]:
food_display_name_list = [str(food_class_dict["display_name"]).lower() for food_class_dict in food_class_dicts]
# for item in food_class_dicts:
food_display_name_list

['almond butter',
 'almonds',
 'green apple',
 'red apple',
 'apricots',
 'artichoke',
 'asparagus',
 'avocado',
 'bacon',
 'bacon and egg burger',
 'bagel',
 'baklava',
 'banana',
 'banana bread',
 'barbecue sauce',
 'basil',
 'bean sprouts',
 'beans',
 'beef curry',
 'diced beef',
 'beef kebab',
 'beef mince',
 'beef patty',
 'roast beef',
 'beef stir-fry',
 'beer',
 'beetroot',
 'biltong',
 'blackberries',
 'blueberries',
 'bok choy',
 'bread',
 'naan bread',
 'broccoli',
 'broccolini',
 'brownies',
 'brussels sprouts',
 'burrito',
 'butter',
 'green cabbage',
 'red cabbage',
 'calamari',
 'candy',
 'green capsicum',
 'orange capsicum',
 'red capsicum',
 'yellow capsicum',
 'carrot',
 'cashews',
 'cauliflower',
 'celery',
 'cheese',
 'cheeseburger',
 'cherries',
 'chicken breast',
 'chicken thighs',
 'chicken wings',
 'chicory',
 'chilli',
 'chimichurri',
 'chives',
 'chocolate',
 'chocolate cake',
 'choko',
 'coconut',
 'coffee',
 'coleslaw',
 'cookies',
 'coriander',
 'corn',
 'co

## Try get some other names for each food class

I'd like to get other names for each food class, for example:
* Eggplant -> Aubergine
* Potato -> Spud
* White Radish -> Daikon Radish

This would enrich the text-based system to have more options at recognizing classes with different names (and wouldn't be limited to just the straight up "display_name").

In [71]:
chat_complete("What are some other names for the food 'beef patty'? Please return them as a Python list only (no extra text). If there are none, please say 'None'.")

<OpenAIObject chat.completion id=chatcmpl-6rGtylcc0gYK3jp5iwnYkzy6UghjZ at 0x7fec119e74a0> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "['burger patty', 'hamburger patty']",
        "role": "assistant"
      }
    }
  ],
  "created": 1678154374,
  "id": "chatcmpl-6rGtylcc0gYK3jp5iwnYkzy6UghjZ",
  "model": "gpt-3.5-turbo-0301",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 13,
    "prompt_tokens": 67,
    "total_tokens": 80
  }
}